In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import os, pickle, itertools
import numpy as np

import smote_variants as sv

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

import sklearn.datasets as datasets
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import utils

In [3]:
oversampler= sv.MulticlassOversampling(sv.polynom_fit_SMOTE())

In [3]:
filepath = '/Users/bogao/DataFiles/new landscape/files/multiclass_independent_data_site.pkl'
with open(filepath, 'rb') as fin:
    data = pickle.load(fin)

In [3]:
data

{'data_train': array([[ 0.    ,  0.    ,  0.    , ..., -3.9349, -3.9349, -4.3219],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    , -1.1231],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    , -1.281 ],
        ...,
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    , -1.3322],
        [ 0.    ,  0.    ,  0.    , ..., -1.2938, -1.2938, -0.8064],
        [ 0.7118,  0.7118,  0.7118, ...,  0.    ,  0.    ,  0.    ]]),
 'data_test': array([[ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    , -4.3219],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ..., -0.0502, -0.0502, -0.0164],
        ...,
        [ 0.2294,  0.2294,  0.2294, ..., -0.0632, -0.0632,  0.    ],
        [ 0.4609,  0.4609,  0.4609, ...,  0.    ,  0.    , -0.0437],
        [ 0.    ,  0.    ,  0.    , ..., -0.2494, -0.2494,  0.    ]]),
 'site_train': array(['Ovary', 'Ovary', 'Ovary', ..., 'Ovary', 'Ovary', 'Ovary'],
       dtype=object),
 'site_test

In [4]:
num_labels = dict([(y,x+1) for x,y in enumerate(set(data['site_train']))])
labels_train = np.array([num_labels[x] for x in data['site_train']])
labels_test = np.array([num_labels[x] for x in data['site_test']])

In [6]:
labels_train

array([1, 1, 1, ..., 1, 1, 1])

In [5]:
model = DecisionTreeClassifier()

In [7]:
X_samp, y_samp= oversampler.sample(data['data_train'], labels_train)

2020-03-17 09:15:06,829:INFO:MulticlassOversampling: Running multiclass oversampling with strategy equalize_1_vs_many_successive
2020-03-17 09:15:06,996:INFO:MulticlassOversampling: Sampling minority class with label: 3
2020-03-17 09:15:07,186:INFO:polynom_fit_SMOTE: Running sampling via ('polynom_fit_SMOTE', "{'proportion': 1.0, 'topology': 'star', 'random_state': None}")
2020-03-17 09:15:07,961:INFO:MulticlassOversampling: Sampling minority class with label: 4
2020-03-17 09:15:08,229:INFO:polynom_fit_SMOTE: Running sampling via ('polynom_fit_SMOTE', "{'proportion': 0.3547786512205213, 'topology': 'star', 'random_state': None}")
2020-03-17 09:15:09,013:INFO:MulticlassOversampling: Sampling minority class with label: 7
2020-03-17 09:15:09,349:INFO:polynom_fit_SMOTE: Running sampling via ('polynom_fit_SMOTE', "{'proportion': 0.2477993488484264, 'topology': 'star', 'random_state': None}")
2020-03-17 09:15:10,380:INFO:MulticlassOversampling: Sampling minority class with label: 8
2020-03-1

In [13]:
X_samp, y_samp = utils.shuffle(X_samp, y_samp)
scores = cross_val_score(model, X_samp, y_samp, cv=5, n_jobs=-1, scoring='f1_macro')

In [12]:
scores

array([0.29673244, 0.72639992, 0.85657088, 0.92693313, 0.90788109])

In [14]:
scores

array([0.81374781, 0.81842444, 0.81856656, 0.81113378, 0.8134606 ])

In [8]:
model_indtest = model.fit(X_samp, y_samp)

In [16]:
y_predict = model_indtest.predict(data['data_test'])

In [17]:
metrics.confusion_matrix(labels_test, y_predict)

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [36, 31, 22, 46, 59, 18, 96, 11, 15, 61],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  2,  0,  0,  4,  0,  0,  0],
       [ 5,  2,  0,  6,  0,  1, 17,  6,  1,  6],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

In [10]:
mela_labels = labels_test[labels_test==5]

In [11]:
mela_test = data['data_test'][labels_test==5]

In [12]:
y_predict = model_indtest.predict(mela_test)

In [13]:
metrics.multilabel_confusion_matrix(mela_labels, y_predict)

array([[[43,  1],
        [ 0,  0]],

       [[41,  3],
        [ 0,  0]],

       [[39,  5],
        [ 0,  0]],

       [[ 0,  0],
        [37,  7]],

       [[38,  6],
        [ 0,  0]],

       [[41,  3],
        [ 0,  0]],

       [[43,  1],
        [ 0,  0]],

       [[26, 18],
        [ 0,  0]]])

In [18]:
metrics.f1_score(mela_labels, y_predict, average='micro')

0.1590909090909091

In [19]:
y_predict

array([ 5, 10,  4,  5, 10, 10,  7,  3, 10,  8, 10,  8,  9, 10,  4,  4, 10,
        8, 10,  7,  4, 10,  4, 10,  3, 10,  7, 10,  5,  7, 10, 10, 10, 10,
        7,  5,  7,  5,  5,  2, 10,  3, 10,  5])

In [6]:
filepath = '/Users/bogao/DataFiles/new landscape/files/multiclass_sites_wo_others.pkl'
with open(filepath, 'rb') as fin:
    data = pickle.load(fin)

In [9]:
num_labels = dict([(y,x+1) for x,y in enumerate(set(data['site']))])
num_labels = np.array([num_labels[x] for x in data['site']])

In [10]:
X_samp, y_samp= oversampler.sample(data['data'], num_labels)

2020-03-17 13:48:55,538:INFO:MulticlassOversampling: Running multiclass oversampling with strategy equalize_1_vs_many_successive
2020-03-17 13:48:55,710:INFO:MulticlassOversampling: Sampling minority class with label: 7
2020-03-17 13:48:55,899:INFO:polynom_fit_SMOTE: Running sampling via ('polynom_fit_SMOTE', "{'proportion': 1.0, 'topology': 'star', 'random_state': None}")
2020-03-17 13:48:56,687:INFO:MulticlassOversampling: Sampling minority class with label: 3
2020-03-17 13:48:56,960:INFO:polynom_fit_SMOTE: Running sampling via ('polynom_fit_SMOTE', "{'proportion': 0.3549586776859504, 'topology': 'star', 'random_state': None}")
2020-03-17 13:48:57,764:INFO:MulticlassOversampling: Sampling minority class with label: 1
2020-03-17 13:48:58,107:INFO:polynom_fit_SMOTE: Running sampling via ('polynom_fit_SMOTE', "{'proportion': 0.2478920741989882, 'topology': 'star', 'random_state': None}")
2020-03-17 13:48:59,148:INFO:MulticlassOversampling: Sampling minority class with label: 4
2020-03-1

In [5]:
filepath = '/Users/bogao/DataFiles/new landscape/files/multiclass_disease_wo_others_none.pkl'
with open(filepath, 'rb') as fin:
    data = pickle.load(fin)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/bogao/DataFiles/new landscape/files/multiclass_sites_wo_others_none.pkl'

In [7]:
num_labels = dict([(y,x+1) for x,y in enumerate(set(data['disease']))])
num_labels = np.array([num_labels[x] for x in data['disease']])

KeyError: 'disease'

In [8]:
X_samp, y_samp= oversampler.sample(data['data'], num_labels)

NameError: name 'num_labels' is not defined

In [11]:
X_samp, y_samp = utils.shuffle(X_samp, y_samp)

In [12]:
x_train, x_test, y_train, y_test = train_test_split(X_samp, y_samp, test_size=0.10, random_state=50)

In [13]:
model = DecisionTreeClassifier()
scores = cross_val_score(model, x_train, y_train, cv=5, n_jobs=-1, scoring='f1_macro')

In [18]:
scores

array([0.80765432, 0.80641677, 0.80977977, 0.80330427, 0.79900236])

In [ ]:
model_indtest = model.fit(x_train, y_train)

In [15]:
y_pred = model_indtest.predict(x_test)

In [17]:
metrics.multilabel_confusion_matrix(y_test, y_pred)

array([[[5859,  125],
        [ 137,  465]],

       [[5828,  111],
        [  97,  550]],

       [[5832,  130],
        [ 143,  481]],

       [[5720,   94],
        [  86,  686]],

       [[5903,   94],
        [ 108,  481]],

       [[5702,  268],
        [ 285,  331]],

       [[5602,  225],
        [ 209,  550]],

       [[5877,   64],
        [  40,  605]],

       [[5869,   85],
        [ 118,  514]],

       [[5795,   91],
        [  64,  636]]])

In [19]:
metrics.confusion_matrix(y_test, y_pred)

array([[465,   9,  10,   8,  10,  48,  35,   2,   8,   7],
       [ 12, 550,   4,  10,  13,  21,  21,   4,   9,   3],
       [ 13,  10, 481,  15,   9,  33,  31,   8,  12,  12],
       [  7,  12,   8, 686,   8,  18,  12,   7,   4,  10],
       [ 10,  14,  13,   8, 481,  26,  21,   7,   3,   6],
       [ 38,  32,  29,  25,  24, 331,  70,  21,  27,  19],
       [ 29,  16,  29,   8,  18,  73, 550,   8,  10,  18],
       [  7,   3,   1,   1,   1,   8,  10, 605,   5,   4],
       [  6,  10,  25,  15,   6,  23,  15,   6, 514,  12],
       [  3,   5,  11,   4,   5,  18,  10,   1,   7, 636]])

In [22]:
metrics.f1_score(y_test, y_pred, average=None)

array([0.78020134, 0.84097859, 0.77894737, 0.88402062, 0.82646048,
       0.54485597, 0.71707953, 0.92085236, 0.83509342, 0.89138052])